# Caribbean Habitat Patch Size (large islands)

This is an indicator for use in Southeast Blueprint 2023. Code by Rua Mordecai.

## Import libraries and define variables

In [21]:
import os
import arcpy

In [22]:
# define spatial reference, workspaces, and source data location
sr= arcpy.SpatialReference(5070)
OutputWorkspace = r"D:\Blueprint\2023\finalIndicatorEdits\CaribbeanBlueprint2023_FinalIndicators\CaribbeanBlueprint2023_FinalIndicators\SpatialData"
landfire = r"D:\Landcover\LF2020_Puerto_Rico_Virgin_Islands_220_IA\LF2020_PRVI_220_IA\LF2020_EVT_220_PRVI\Tif\LV20_EVT_220.tif"
landfireRoads = r"D:\Landcover\LF2020_Puerto_Rico_Virgin_Islands_220_IA\LF2020_PRVI_220_IA\LF2020_Roads_220_PRVI\Tif\LV20_Roads_220.tif"

In [23]:
# define final output name
IndicatorFileName = "CaribbeanHabitatPatchSizeLargeIslands.tif"

In [24]:
# define raster used for cell size, extent, and snapping
CaribbeanRaster= r"D:\Blueprint\2023\extent\VIPR_Extent_v6.tif"

In [25]:
# If looking at the impact of restoration in parcel, define the shapefile where all landcover should become natural
# RestoreArea = r"D:\UserSupport\USVI_RLA2023\ParcelRLA\ParcelRLA.shp"

In [26]:
# island boundary data. This is for islands size and to separate ocean from fresh and brackish water
islandBoundary = r"D:\Blueprint\2023\extent\Islands3Size.tif"

In [27]:
# Sometimes arcpro is fussy about not overwriting things
arcpy.env.overwriteOutput = True

## Start analysis

In [28]:
# Set the source workspace for panther data
arcpy.env.workspace = OutputWorkspace

In [36]:
# Reclassify Landfire with natural as 1 and everything else NODATA. All classes in “EVT_NAME” that start with “Quarries”,
# “Developed”, or “Agriculture” are considered unnatural. “Caribbean bush fruit and berries”, which captures sun coffee plantations,
# is also considered unnatural. “Tropical agroforestry plantation”, which includes shade coffee is considered natural. 
# Water is classified as 1.
with arcpy.EnvManager(outputCoordinateSystem=sr, extent=CaribbeanRaster, snapRaster=CaribbeanRaster, cellSize=CaribbeanRaster):
    out_raster = arcpy.sa.Reclassify(landfire, "Value", "-9999 NODATA;7292 1;7295 7296 NODATA;7297 7298 NODATA;7299 7300 NODATA;7754 NODATA;7755 NODATA;7838 NODATA;7861 7888 1;9303 9803 1", "DATA"); out_raster.save("naturalOW.tif")

In [37]:
# prep islands boundary raster to remove ocean. Reclass so all islands have same value
with arcpy.EnvManager(outputCoordinateSystem=sr, extent=CaribbeanRaster, snapRaster=CaribbeanRaster, cellSize=CaribbeanRaster):
    out_raster = arcpy.sa.Reclassify(islandBoundary, "Value", "1 NODATA;2 4 1", "DATA"); out_raster.save("islandBoundaryBin.tif")

In [38]:
# Combine rasters to make ocean noData
out_raster = arcpy.sa.Times("naturalOW.tif", "islandBoundaryBin.tif"); out_raster.save("natural.tif")

In [39]:
# Reclassify Landfire roads with primary, secondary, and tertiary roads as NODATA. 
# Thinned road and other land pixels are 1. Thinned roads tend to get little to no traffic in the Caribbean and can look relatively natural 
with arcpy.EnvManager(outputCoordinateSystem=sr, extent=CaribbeanRaster, snapRaster=CaribbeanRaster, cellSize=CaribbeanRaster):
    out_raster = arcpy.sa.Reclassify(landfireRoads, "Value", "-9999 NODATA;0 1;20 22 NODATA;23 1", "DATA"); out_raster.save("noRoad.tif")

### Uncomment this section if looking at impact of restoration

In [13]:
# Make a copy of restoration shapefile
# arcpy.management.CopyFeatures(RestoreArea, "restoreArea.shp")

In [14]:
# add a field to convert to raster
# arcpy.management.CalculateField("restoreArea.shp", "raster", "1", "PYTHON3", '', "SHORT")

In [15]:
# convert to raster and snap to landfire
# with arcpy.EnvManager(outputCoordinateSystem=landfire, extent=landfire, snapRaster=landfire, cellSize=landfire):
#    arcpy.conversion.FeatureToRaster(in_features="restoreArea.shp", field="raster", out_raster="restoreAreaR.tif", cell_size=landfire)

In [16]:
# Change noData pixels to 0  
# out_raster = arcpy.sa.Reclassify("restoreAreaR.tif", "Value", "NODATA 0;1 1", "DATA"); out_raster.save("restoreAreaRZ.tif")

In [17]:
# Copy raster so we can use it below but get back to the original name
# arcpy.management.CopyRaster("natural.tif","natural2.tif")

In [18]:
# change natural raster so anything in restoration area that's not natural becomes natural
# out_raster = arcpy.sa.Con(arcpy.sa.Raster("restoreAreaRZ.tif") > 0,1,"natural2.tif"); out_raster.save("natural.tif")

### Analysis continues

In [40]:
# Combine rasters. Skip this line if doing restoration analysis
out_raster = arcpy.sa.Times("natural.tif", "noRoad.tif"); out_raster.save("naturalNoRoad.tif")

In [41]:
# figure out the size of groups that each pixels belong to
out_raster = arcpy.sa.RegionGroup("NaturalNoRoad.tif"); out_raster.save("NaturalNoRoad_regionGroup.tif")

In [42]:
# find patches greater than 449 pixels(about 100 acres). Technically 45 pixels is about 10.01 acres
out_raster = arcpy.sa.Con("NaturalNoRoad_regionGroup.tif", 1, 0, "Count > 45"); out_raster.save("NaturalNoRoad_regionGroup_gtr45.tif")

In [43]:
# find patches greater than 449 pixels(about 100 acres). Technically 449 pixels is about 99.85 acres
out_raster = arcpy.sa.Con("NaturalNoRoad_regionGroup.tif", 1, 0, "Count > 449"); out_raster.save("NaturalNoRoad_regionGroup_gtr449.tif")

In [44]:
# find patches greater than 4496 pixels(about 1000 acres). Technically 4496 is about 999.89 acres
out_raster = arcpy.sa.Con("NaturalNoRoad_regionGroup.tif", 1, 0, "Count > 4496"); out_raster.save("NaturalNoRoad_regionGroup_gtr4496.tif")

In [45]:
# find patches greater than 44965 pixels(about 10000 acres). Technically 44965 is about 999.98 acres
out_raster = arcpy.sa.Con("NaturalNoRoad_regionGroup.tif", 1, 0, "Count > 44965"); out_raster.save("NaturalNoRoad_regionGroup_gtr44965.tif")

In [46]:
# add rasters together
output_raster = arcpy.ia.RasterCalculator(["NaturalNoRoad_regionGroup_gtr44965.tif","NaturalNoRoad_regionGroup_gtr4496.tif","NaturalNoRoad_regionGroup_gtr449.tif","NaturalNoRoad_regionGroup_gtr45.tif"],['a','b','c','d'],'a + b + c + d'); output_raster.save("PatchBins.tif")

In [47]:
# Change noData pixels to 0 so we can separate nonnatural on land and ocean. Add 1 to everything to shift the legend 
out_raster = arcpy.sa.Reclassify("PatchBins.tif", "Value", "NODATA 0;0 1;1 2;2 3;3 4;4 5", "DATA"); out_raster.save("PatchNoData.tif")

In [48]:
# Remove ocean
out_raster = arcpy.sa.Times("PatchNoData.tif", "islandBoundaryBin.tif"); out_raster.save("PatchBinsClean.tif")

In [49]:
# make mask for large islands
out_raster = arcpy.sa.Reclassify(islandBoundary, "Value", "1 NODATA;2 3 NODATA;4 1", "DATA"); out_raster.save("LargeIslandMask.tif")

In [50]:
# Reduce to just islands for this indicator. Technically this includes both "small" and "medium" islands from the island boundary data
out_raster = arcpy.sa.Times("LargeIslandMask.tif", "PatchBinsClean.tif"); out_raster.save("largeIslandBins.tif")

In [51]:
# export nonclipped version of indicator
#with arcpy.EnvManager(outputCoordinateSystem=sr, extent=CaribbeanRaster, snapRaster=CaribbeanRaster, cellSize=CaribbeanRaster):
#    arcpy.management.CopyRaster("PatchBinsClean.tif", IndicatorFileName, '', None, "255", "NONE", "NONE", "8_BIT_UNSIGNED", "NONE", "NONE", "TIFF", "NONE", "CURRENT_SLICE", "NO_TRANSPOSE")

In [52]:
# clip raster to Caribbean extent
with arcpy.EnvManager(outputCoordinateSystem=sr, extent=CaribbeanRaster, snapRaster=CaribbeanRaster, cellSize=CaribbeanRaster):
    out_raster = arcpy.sa.ExtractByMask("largeIslandBins.tif", CaribbeanRaster); out_raster.save(IndicatorFileName)

## Finalize indicator

In [53]:
# set code block for legend
codeblock = """
def Reclass(value):
    if value == 0:
        return '0 = Developed or agriculture'    
    elif value == 1:
        return '1 = Very small patch (≤10 acres)'
    elif value == 2:
        return '2 = Small patch (>10-100 acres)'
    elif value == 3:
        return '3 = Medium patch (>100-1,000 acres)' 
    elif value == 4:
        return '4 = Large patch (>1,000-10,000 acres)'
    elif value == 5:
        return '5 = Very large patch (>10,000 acres)'
    
"""

In [54]:
# add and calculate description field to hold indicator values
arcpy.management.CalculateField(IndicatorFileName, "descript", "Reclass(!value!)", "PYTHON3", codeblock, "TEXT")

<Result 'D:\\Blueprint\\2023\\Caribbean\\Indicators\\LandscapeIntegrity\\working\\CaribbeanHabitatPatchSizeLargeIslands.tif'>

In [29]:
# set code block for next step
codeblock = """
def Reclass1(Value):
	if Value == 5:
		return 0
	if Value == 4:
		return 16
	if Value == 3:
		return 62
	if Value == 2:
		return 121
	if Value == 1:
		return 195
	if Value == 0:
		return 255
	else:
		return 255
		
def Reclass2(Value):
	if Value == 5:
		return 68
	if Value == 4:
		return 123
	if Value == 3:
		return 171
	if Value == 2:
		return 202
	if Value == 1:
		return 232
	if Value == 0:
		return 255
	else:
		return 255
		
def Reclass3(Value):
	if Value == 5:
		return 27
	if Value == 4:
		return 55
	if Value == 3:
		return 114
	if Value == 2:
		return 177
	if Value == 1:
		return 225
	if Value == 0:
		return 255
	else:
		return 255
		
"""

In [30]:
# calculate Red field
arcpy.management.CalculateField(IndicatorFileName, "Red", "Reclass1(!Value!)", "PYTHON3", codeblock, "SHORT")
# calculate Green field
arcpy.management.CalculateField(IndicatorFileName, "Green", "Reclass2(!Value!)", "PYTHON3", codeblock, "SHORT")
# calculate Blue field
arcpy.management.CalculateField(IndicatorFileName, "Blue", "Reclass3(!Value!)", "PYTHON3", codeblock, "SHORT")

<Result 'D:\\Blueprint\\2023\\finalIndicatorEdits\\CaribbeanBlueprint2023_FinalIndicators\\CaribbeanBlueprint2023_FinalIndicators\\SpatialData\\CaribbeanHabitatPatchSizeLargeIslands.tif'>